# Ingeniería de Variables

__Carga del dataset__

In [10]:
import numpy as np
import pandas as pd
import os
from functools import reduce

In [11]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [12]:
ruta = '../data/ecobici/viajes.parquet'
df = pd.read_parquet(ruta)
df.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Ciclo_EstacionArribo,fh_retiro,fh_arribo
0,M,29,5210661,134,147,2023-09-30 23:55:54,2023-10-01 00:00:16
1,M,26,8316747,045,147,2023-09-30 23:46:57,2023-10-01 00:00:32
2,M,30,3749272,384,337,2023-09-30 23:50:50,2023-10-01 00:00:34
3,F,40,2961233,014,018,2023-09-30 23:50:24,2023-10-01 00:00:36
4,F,34,2661924,155,139,2023-09-30 23:55:48,2023-10-01 00:00:39


In [13]:
df.dtypes

Genero_Usuario                   object
Edad_Usuario                      int32
Bici                              int32
Ciclo_Estacion_Retiro            object
Ciclo_EstacionArribo             object
fh_retiro                datetime64[ns]
fh_arribo                datetime64[ns]
dtype: object

In [14]:
df['Genero_Usuario'].value_counts()

Genero_Usuario
M    23542761
F     9615995
O      689703
?       45017
Name: count, dtype: int64

__Ingeniería de variables__

- Unidad Muestral: Ciclo Estación en una fecha en una hora específica.
- Generar Variables
- Matriz de predictoras
- Variables objetivo
- Ventanas de Tiempo
- TAD

In [15]:
# unidad muestral
um = ['Ciclo_Estacion_Retiro', 'hrs']
df['hrs'] = df['fh_retiro'].map( lambda x: x.strftime('%Y%m%d%H') ).astype(int)
df.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Ciclo_EstacionArribo,fh_retiro,fh_arribo,hrs
0,M,29,5210661,134,147,2023-09-30 23:55:54,2023-10-01 00:00:16,2023093023
1,M,26,8316747,045,147,2023-09-30 23:46:57,2023-10-01 00:00:32,2023093023
2,M,30,3749272,384,337,2023-09-30 23:50:50,2023-10-01 00:00:34,2023093023
3,F,40,2961233,014,018,2023-09-30 23:50:24,2023-10-01 00:00:36,2023093023
4,F,34,2661924,155,139,2023-09-30 23:55:48,2023-10-01 00:00:39,2023093023


In [16]:
# Agrupar los viajes para la demanda de ecobicis por estacion de retiro en cada fecha-hora
df_g = df.groupby( um )['Bici'].count().reset_index()
df_g.sample(10)

,Ciclo_Estacion_Retiro,hrs,Bici
1024010,085,2024050417,8
1751944,150,2024031417,24
2700787,243,2023122418,1
3418157,316,2025011116,3
1488654,128,2024121514,15
1422636,123,2024020216,13
4103534,393,2024081713,9
6017349,651,2024122319,1
2598324,231,2024042818,4
645544,054,2023092606,2


### Definición de variables

$$y = H(X)$$

- Promedio de la edad en Ciclo estación fecha hora (por unidad muestral)
- Porcentaje de hombres por UM
- Número de hombres
- Número de mujeres
- Promedio de minutos de los viajes a nivel UM.
- Promedio, max, min, desv, ...

__Nomenclaturas de variables__

* v_ -> númericas
* c_ -> categóricas
* fh_ -> fechas
* bl_ -> Booleanas

### Construcción TAD

__Variable Objetivo__

In [17]:
df_g.rename( columns={'Bici':'v_num_viajes'}, inplace=True )
df_g.head()

,Ciclo_Estacion_Retiro,hrs,v_num_viajes
0,001,2023030105,2
1,001,2023030106,3
2,001,2023030107,4
3,001,2023030108,15
4,001,2023030109,13


__Promedio de la Edad__

In [18]:
# promedio de la edad por unidad muestral
df_prom_edad = df.groupby( um )['Edad_Usuario'].mean().reset_index()
df_prom_edad.rename( columns={'Edad_Usuario':'v_edad_prom'}, inplace=True )
df_prom_edad.head(5)

,Ciclo_Estacion_Retiro,hrs,v_edad_prom
0,001,2023030105,34.500000
1,001,2023030106,39.666667
2,001,2023030107,44.500000
3,001,2023030108,38.200000
4,001,2023030109,34.153846


__Numeralia de genero__

In [19]:
# Reemplazamos la etiqueta ? en la columna genero por la etiqueta 'No Disponible'
df['Genero_Usuario'] = df['Genero_Usuario'].replace('?', 'ND')

# Reemplazamos la etiqueta O por la etiqueta 'No Disponible'
df['Genero_Usuario'] = df['Genero_Usuario'].replace('O', 'ND')

df['Genero_Usuario'].value_counts()

Genero_Usuario
M     23542761
F      9615995
ND      734720
Name: count, dtype: int64

In [20]:
aux = df.pivot_table( index=um, columns='Genero_Usuario', values='Bici', aggfunc='count', fill_value=0 )
aux.head()

Genero_Usuario                    F   M  ND
Ciclo_Estacion_Retiro hrs                  
001                   2023030105  0   2   0
                      2023030106  3   0   0
                      2023030107  1   3   0
                      2023030108  4  10   1
                      2023030109  3  10   0

In [21]:
aux['v_personas_tot'] = aux.sum(axis=1)
aux.head()

Genero_Usuario                    F   M  ND  v_personas_tot
Ciclo_Estacion_Retiro hrs                                  
001                   2023030105  0   2   0               2
                      2023030106  3   0   0               3
                      2023030107  1   3   0               4
                      2023030108  4  10   1              15
                      2023030109  3  10   0              13

In [22]:
aux.columns = ['v_' + c + '_tot' if c != 'v_personas_tot' else c for c in aux.columns]
aux.head()

v_F_tot  v_M_tot  v_ND_tot  v_personas_tot
Ciclo_Estacion_Retiro hrs                                                   
001                   2023030105        0        2         0               2
                      2023030106        3        0         0               3
                      2023030107        1        3         0               4
                      2023030108        4       10         1              15
                      2023030109        3       10         0              13

In [23]:
aux['v_pct_F'] = np.where( aux['v_personas_tot'] == 0 , 0, aux['v_F_tot'] / aux['v_personas_tot'] )
aux['v_pct_M'] = np.where( aux['v_personas_tot'] == 0 , 0, aux['v_M_tot'] / aux['v_personas_tot'] )
aux['v_pct_M'] = np.where( aux['v_personas_tot'] == 0 , 0, aux['v_ND_tot'] / aux['v_personas_tot'] )
aux.head()

v_F_tot  v_M_tot  v_ND_tot  v_personas_tot  \
Ciclo_Estacion_Retiro hrs                                                      
001                   2023030105        0        2         0               2   
                      2023030106        3        0         0               3   
                      2023030107        1        3         0               4   
                      2023030108        4       10         1              15   
                      2023030109        3       10         0              13   

                                   v_pct_F   v_pct_M  
Ciclo_Estacion_Retiro hrs                             
001                   2023030105  0.000000  0.000000  
                      2023030106  1.000000  0.000000  
                      2023030107  0.250000  0.000000  
                      2023030108  0.266667  0.066667  
                      2023030109  0.230769  0.000000

In [24]:
df_pct_genero = aux.reset_index()
aux.head()

v_F_tot  v_M_tot  v_ND_tot  v_personas_tot  \
Ciclo_Estacion_Retiro hrs                                                      
001                   2023030105        0        2         0               2   
                      2023030106        3        0         0               3   
                      2023030107        1        3         0               4   
                      2023030108        4       10         1              15   
                      2023030109        3       10         0              13   

                                   v_pct_F   v_pct_M  
Ciclo_Estacion_Retiro hrs                             
001                   2023030105  0.000000  0.000000  
                      2023030106  1.000000  0.000000  
                      2023030107  0.250000  0.000000  
                      2023030108  0.266667  0.066667  
                      2023030109  0.230769  0.000000

__Tiempo de viajes__

In [25]:
df['v_tiempo_viaje'] = (df['fh_arribo'] - df['fh_retiro']).dt.total_seconds() / 60
df.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Ciclo_EstacionArribo,fh_retiro,fh_arribo,hrs,v_tiempo_viaje
0,M,29,5210661,134,147,2023-09-30 23:55:54,2023-10-01 00:00:16,2023093023,4.366667
1,M,26,8316747,045,147,2023-09-30 23:46:57,2023-10-01 00:00:32,2023093023,13.583333
2,M,30,3749272,384,337,2023-09-30 23:50:50,2023-10-01 00:00:34,2023093023,9.733333
3,F,40,2961233,014,018,2023-09-30 23:50:24,2023-10-01 00:00:36,2023093023,10.200000
4,F,34,2661924,155,139,2023-09-30 23:55:48,2023-10-01 00:00:39,2023093023,4.850000


In [26]:
df_tiempos = df.groupby( um ).agg( { 'v_tiempo_viaje': ['min', 'max', 'mean', 'std'] } ).reset_index()
df_tiempos.head()

Ciclo_Estacion_Retiro         hrs v_tiempo_viaje                        \
                                               min        max       mean   
0                   001  2023030105       4.916667   5.966667   5.441667   
1                   001  2023030106       4.083333  16.683333   9.372222   
2                   001  2023030107       5.366667  31.616667  15.754167   
3                   001  2023030108       4.416667  42.066667  13.632222   
4                   001  2023030109       4.133333  27.566667  12.012821   

              
         std  
0   0.742462  
1   6.538887  
2  11.183721  
3  10.312965  
4   7.224816

In [27]:
df_tiempos.columns

MultiIndex([('Ciclo_Estacion_Retiro',     ''),
            (                  'hrs',     ''),
            (       'v_tiempo_viaje',  'min'),
            (       'v_tiempo_viaje',  'max'),
            (       'v_tiempo_viaje', 'mean'),
            (       'v_tiempo_viaje',  'std')],
           )

In [28]:
df_tiempos.columns = [c[0]+'_'+c[1] if c[1] != '' else c[0] for c in df_tiempos.columns]
df_tiempos.head()

,Ciclo_Estacion_Retiro,hrs,v_tiempo_viaje_min,v_tiempo_viaje_max,v_tiempo_viaje_mean,v_tiempo_viaje_std
0,001,2023030105,4.916667,5.966667,5.441667,0.742462
1,001,2023030106,4.083333,16.683333,9.372222,6.538887
2,001,2023030107,5.366667,31.616667,15.754167,11.183721
3,001,2023030108,4.416667,42.066667,13.632222,10.312965
4,001,2023030109,4.133333,27.566667,12.012821,7.224816


__Nivel de Demanda__

* <=5 := BAJA
* 5> <=10 := MEDIA
* 10> := ALTA

In [29]:
def func_demanda(num_viajes):

    if num_viajes <= 5:
        r = "BAJA"
    elif num_viajes <= 10:
        r = "MEDIA"
    elif num_viajes > 10:
        r = "ALTA"
    else:
        r = "ERROR"
    
    return r

In [30]:
df_g['c_nivel_demanda'] = df_g['v_num_viajes'].map(lambda x: func_demanda(x) )
df_g.head()

,Ciclo_Estacion_Retiro,hrs,v_num_viajes,c_nivel_demanda
0,001,2023030105,2,BAJA
1,001,2023030106,3,BAJA
2,001,2023030107,4,BAJA
3,001,2023030108,15,ALTA
4,001,2023030109,13,ALTA


### Matriz de predictoras

In [31]:
X_ = reduce( lambda x, y: pd.merge(x, y, on=um), [df_g, df_prom_edad, df_pct_genero, df_tiempos] )
X_.head()

,Ciclo_Estacion_Retiro,hrs,v_num_viajes,c_nivel_demanda,v_edad_prom,v_F_tot,v_M_tot,v_ND_tot,v_personas_tot,v_pct_F,v_pct_M,v_tiempo_viaje_min,v_tiempo_viaje_max,v_tiempo_viaje_mean,v_tiempo_viaje_std
0,001,2023030105,2,BAJA,34.500000,0,2,0,2,0.000000,0.000000,4.916667,5.966667,5.441667,0.742462
1,001,2023030106,3,BAJA,39.666667,3,0,0,3,1.000000,0.000000,4.083333,16.683333,9.372222,6.538887
2,001,2023030107,4,BAJA,44.500000,1,3,0,4,0.250000,0.000000,5.366667,31.616667,15.754167,11.183721
3,001,2023030108,15,ALTA,38.200000,4,10,1,15,0.266667,0.066667,4.416667,42.066667,13.632222,10.312965
4,001,2023030109,13,ALTA,34.153846,3,10,0,13,0.230769,0.000000,4.133333,27.566667,12.012821,7.224816


In [32]:
X_.rename( columns={'v_num_viajes':'y'}, inplace=True )
X_.to_parquet('../data/ecobici/TAD.parquet', index=False)

In [33]:
cols_X = [ c for c in X_.columns if c not in ['y'] ]
X_[cols_X]

,Ciclo_Estacion_Retiro,hrs,c_nivel_demanda,v_edad_prom,v_F_tot,v_M_tot,v_ND_tot,v_personas_tot,v_pct_F,v_pct_M,v_tiempo_viaje_min,v_tiempo_viaje_max,v_tiempo_viaje_mean,v_tiempo_viaje_std
0,001,2023030105,BAJA,34.500000,0,2,0,2,0.000000,0.000000,4.916667,5.966667,5.441667,0.742462
1,001,2023030106,BAJA,39.666667,3,0,0,3,1.000000,0.000000,4.083333,16.683333,9.372222,6.538887
2,001,2023030107,BAJA,44.500000,1,3,0,4,0.250000,0.000000,5.366667,31.616667,15.754167,11.183721
3,001,2023030108,ALTA,38.200000,4,10,1,15,0.266667,0.066667,4.416667,42.066667,13.632222,10.312965
4,001,2023030109,ALTA,34.153846,3,10,0,13,0.230769,0.000000,4.133333,27.566667,12.012821,7.224816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6369325,CE-642,2023113014,BAJA,36.000000,1,0,0,1,1.000000,0.000000,16.350000,16.350000,16.350000,NaN
6369326,CE-642,2023113019,BAJA,36.500000,1,1,0,2,0.500000,0.000000,8.216667,12.300000,10.258333,2.887353
6369327,CE-642,2023113020,BAJA,40.000000,0,1,0,1,0.000000,0.000000,14.566667,14.566667,14.566667,NaN
6369328,CE-642,2023113022,BAJA,34.000000,0,1,0,1,0.000000,0.000000,30.183333,30.183333,30.183333,NaN


In [34]:
# Unidad Muestral (um) + Variable objetio (y)
X_[um + ['y']]

,Ciclo_Estacion_Retiro,hrs,y
0,001,2023030105,2
1,001,2023030106,3
2,001,2023030107,4
3,001,2023030108,15
4,001,2023030109,13
...,...,...,...
6369325,CE-642,2023113014,1
6369326,CE-642,2023113019,2
6369327,CE-642,2023113020,1
6369328,CE-642,2023113022,1


### Catalogo del Tiempo

In [35]:
X_.head()

,Ciclo_Estacion_Retiro,hrs,y,c_nivel_demanda,v_edad_prom,v_F_tot,v_M_tot,v_ND_tot,v_personas_tot,v_pct_F,v_pct_M,v_tiempo_viaje_min,v_tiempo_viaje_max,v_tiempo_viaje_mean,v_tiempo_viaje_std
0,001,2023030105,2,BAJA,34.500000,0,2,0,2,0.000000,0.000000,4.916667,5.966667,5.441667,0.742462
1,001,2023030106,3,BAJA,39.666667,3,0,0,3,1.000000,0.000000,4.083333,16.683333,9.372222,6.538887
2,001,2023030107,4,BAJA,44.500000,1,3,0,4,0.250000,0.000000,5.366667,31.616667,15.754167,11.183721
3,001,2023030108,15,ALTA,38.200000,4,10,1,15,0.266667,0.066667,4.416667,42.066667,13.632222,10.312965
4,001,2023030109,13,ALTA,34.153846,3,10,0,13,0.230769,0.000000,4.133333,27.566667,12.012821,7.224816


In [36]:
rango_fechas = pd.date_range(start=pd.to_datetime(X_['hrs'].min(), format='%Y%m%d%H'),
                             end=pd.to_datetime(X_['hrs'].max(), format='%Y%m%d%H'))

rango_fechas

DatetimeIndex(['2022-08-19 14:00:00', '2022-08-20 14:00:00',
               '2022-08-21 14:00:00', '2022-08-22 14:00:00',
               '2022-08-23 14:00:00', '2022-08-24 14:00:00',
               '2022-08-25 14:00:00', '2022-08-26 14:00:00',
               '2022-08-27 14:00:00', '2022-08-28 14:00:00',
               ...
               '2025-01-22 14:00:00', '2025-01-23 14:00:00',
               '2025-01-24 14:00:00', '2025-01-25 14:00:00',
               '2025-01-26 14:00:00', '2025-01-27 14:00:00',
               '2025-01-28 14:00:00', '2025-01-29 14:00:00',
               '2025-01-30 14:00:00', '2025-01-31 14:00:00'],
              dtype='datetime64[ns]', length=897, freq='D')

In [37]:
cat_hrs = pd.DataFrame(rango_fechas, columns=['hrs'])
cat_hrs['hrs'] = cat_hrs['hrs'].dt.strftime('%Y%m%d%H').astype(int)
cat_hrs['hrs'].head()

0    2022081914
1    2022082014
2    2022082114
3    2022082214
4    2022082314
Name: hrs, dtype: int64

In [38]:
cat_hrs.index

RangeIndex(start=0, stop=897, step=1)

In [39]:
cat_hrs['ancla'] = cat_hrs.index+1

In [40]:
cat_hrs.dtypes

hrs      int64
ancla    int64
dtype: object

In [41]:
cat_hrs

,hrs,ancla
0,2022081914,1
1,2022082014,2
2,2022082114,3
3,2022082214,4
4,2022082314,5
...,...,...
892,2025012714,893
893,2025012814,894
894,2025012914,895
895,2025013014,896


In [42]:
X_.shape

(6369330, 15)

In [43]:
aux.shape, df_prom_edad.shape, df_pct_genero.shape, df_tiempos.shape, df_g.shape

((6369330, 6), (6369330, 3), (6369330, 8), (6369330, 6), (6369330, 4))

In [44]:
X_ = X_.merge( cat_hrs, on=['hrs'], how='inner').reset_index(drop=True)

In [45]:
X_.drop('hrs', axis=1, inplace=True)

In [46]:
X_.shape

(379058, 15)

In [47]:
aux.shape, df_prom_edad.shape, df_pct_genero.shape, df_tiempos.shape, df_g.shape

((6369330, 6), (6369330, 3), (6369330, 8), (6369330, 6), (6369330, 4))

In [48]:
%whos

Variable        Type                          Data/Info
-------------------------------------------------------
X_              DataFrame                            Ciclo_Estacion_Ret<...>379058 rows x 15 columns]
aux             DataFrame                                              <...>6369330 rows x 6 columns]
cat_hrs         DataFrame                                 hrs  ancla\n0<...>n\n[897 rows x 2 columns]
cols_X          list                          n=14
df              DataFrame                             Genero_Usuario  E<...>3893476 rows x 9 columns]
df_g            DataFrame                             Ciclo_Estacion_Re<...>6369330 rows x 4 columns]
df_pct_genero   DataFrame                             Ciclo_Estacion_Re<...>6369330 rows x 8 columns]
df_prom_edad    DataFrame                             Ciclo_Estacion_Re<...>6369330 rows x 3 columns]
df_tiempos      DataFrame                             Ciclo_Estacion_Re<...>6369330 rows x 6 columns]
func_demanda    funct

In [49]:
del aux
del df_prom_edad
del df_pct_genero
del df_tiempos
del df_g

In [50]:
import gc
gc.collect()

18

In [51]:
%whos

Variable       Type                          Data/Info
------------------------------------------------------
X_             DataFrame                            Ciclo_Estacion_Ret<...>379058 rows x 15 columns]
cat_hrs        DataFrame                                 hrs  ancla\n0<...>n\n[897 rows x 2 columns]
cols_X         list                          n=14
df             DataFrame                             Genero_Usuario  E<...>3893476 rows x 9 columns]
func_demanda   function                      <function func_demanda at 0x10cca1b20>
gc             module                        <module 'gc' (built-in)>
np             module                        <module 'numpy' from '/Us<...>kages/numpy/__init__.py'>
os             module                        <module 'os' (frozen)>
pd             module                        <module 'pandas' from '/U<...>ages/pandas/__init__.py'>
rango_fechas   DatetimeIndex                 DatetimeIndex(['2022-08-1<...>]', length=897, freq='D')
reduce      

### Ventanas de tiempo

In [52]:
vdes = 1   # Desplazamiento
vobs = 100 # numero de observaciones

In [53]:
hr_i = cat_hrs['ancla'].min() # Ventana inicial
hr_f = cat_hrs['ancla'].max() # Ventana final

In [54]:
hr_i, hr_f

(np.int64(1), np.int64(897))

In [55]:
ancla_i = hr_i + vobs + 1 # ventana incial
ancla_f = hr_f + vdes

In [56]:
ancla_i, ancla_f

(np.int64(102), np.int64(898))

In [57]:
print("Ventanas: ", vdes, vobs)
print("Horas: ", hr_i, hr_f)
print("Anclas: ", ancla_i, ancla_f)

Ventanas:  1 100
Horas:  1 897
Anclas:  102 898


In [58]:
lst_var = X_.filter(like='v_').columns.tolist()
lst_var

['v_edad_prom',
 'v_F_tot',
 'v_M_tot',
 'v_ND_tot',
 'v_personas_tot',
 'v_pct_F',
 'v_pct_M',
 'v_tiempo_viaje_min',
 'v_tiempo_viaje_max',
 'v_tiempo_viaje_mean',
 'v_tiempo_viaje_std']

In [59]:
X_['Ciclo_Estacion_Retiro'].value_counts()

Ciclo_Estacion_Retiro
001        687
019        687
150        687
136        687
111        687
          ... 
676        170
681        158
524        117
4422222      3
CE-642       2
Name: count, Length: 679, dtype: int64

In [62]:
def ventanas( df_aux, variable, ancla ):
    # ancla = 3010000
    # variable = 'v_tiempo_viaje_mean'
    # print("De la hora: ",(ancla - vobs + 1 ) ,", a: " ,ancla )
    df_aux = df_aux.loc[ (df_aux['ancla'] >= (ancla - vobs + 1 )) & ( df_aux['ancla'] <= ancla )  ].reset_index(drop=True)
    df_aux = df_aux.pivot_table( index='Ciclo_Estacion_Retiro' , values= variable , aggfunc=['sum','mean','min','max','std'] )
    df_aux.columns = [f'{b}_{a}_{vobs}' for a,b in df_aux.columns]
    df_aux.insert(0,'ancla',ancla)
    return df_aux.reset_index()

### Matriz de predictoras

In [63]:
ancla_i, ancla_f

(np.int64(102), np.int64(898))

In [64]:
%%time
X = pd.concat( map( lambda ancla: ventanas( X_ , lst_var , ancla ) , range(ancla_i, ancla_f+1) ), ignore_index=True )

CPU times: user 12.8 s, sys: 1.65 s, total: 14.5 s
Wall time: 14.6 s


In [65]:
pd.set_option('display.max_columns', 200)

In [66]:
X.shape

(416749, 57)

In [67]:
X.dtypes

Ciclo_Estacion_Retiro            object
ancla                             int64
v_F_tot_sum_100                 float64
v_M_tot_sum_100                 float64
v_ND_tot_sum_100                float64
v_edad_prom_sum_100             float64
v_pct_F_sum_100                 float64
v_pct_M_sum_100                 float64
v_personas_tot_sum_100          float64
v_tiempo_viaje_max_sum_100      float64
v_tiempo_viaje_mean_sum_100     float64
v_tiempo_viaje_min_sum_100      float64
v_tiempo_viaje_std_sum_100      float64
v_F_tot_mean_100                float64
v_M_tot_mean_100                float64
v_ND_tot_mean_100               float64
v_edad_prom_mean_100            float64
v_pct_F_mean_100                float64
v_pct_M_mean_100                float64
v_personas_tot_mean_100         float64
v_tiempo_viaje_max_mean_100     float64
v_tiempo_viaje_mean_mean_100    float64
v_tiempo_viaje_min_mean_100     float64
v_F_tot_min_100                 float64
v_M_tot_min_100                 float64


In [68]:
# Para disminuir un poco la memoria
X['ancla'] = X['ancla'].astype('int32')

In [69]:
X_.to_parquet('../data/ecobici/X_ecobici.parquet', index=False)

In [70]:
X

,Ciclo_Estacion_Retiro,ancla,v_F_tot_sum_100,v_M_tot_sum_100,v_ND_tot_sum_100,v_edad_prom_sum_100,v_pct_F_sum_100,v_pct_M_sum_100,v_personas_tot_sum_100,v_tiempo_viaje_max_sum_100,v_tiempo_viaje_mean_sum_100,v_tiempo_viaje_min_sum_100,v_tiempo_viaje_std_sum_100,v_F_tot_mean_100,v_M_tot_mean_100,v_ND_tot_mean_100,v_edad_prom_mean_100,v_pct_F_mean_100,v_pct_M_mean_100,v_personas_tot_mean_100,v_tiempo_viaje_max_mean_100,v_tiempo_viaje_mean_mean_100,v_tiempo_viaje_min_mean_100,v_F_tot_min_100,v_M_tot_min_100,v_ND_tot_min_100,v_edad_prom_min_100,v_pct_F_min_100,v_pct_M_min_100,v_personas_tot_min_100,v_tiempo_viaje_max_min_100,v_tiempo_viaje_mean_min_100,v_tiempo_viaje_min_min_100,v_F_tot_max_100,v_M_tot_max_100,v_ND_tot_max_100,v_edad_prom_max_100,v_pct_F_max_100,v_pct_M_max_100,v_personas_tot_max_100,v_tiempo_viaje_max_max_100,v_tiempo_viaje_mean_max_100,v_tiempo_viaje_min_max_100,v_tiempo_viaje_std_mean_100,v_tiempo_viaje_std_min_100,v_tiempo_viaje_std_max_100,v_F_tot_std_100,v_M_tot_std_100,v_ND_tot_std_100,v_edad_prom_std_100,v_pct_F_std_100,v_pct_M_std_100,v_personas_tot_std_100,v_tiempo_viaje_max_std_100,v_tiempo_viaje_mean_std_100,v_tiempo_viaje_min_std_100,v_tiempo_viaje_std_std_100
0,284,170,0.0,1.0,0.0,28.000000,0.000000,0.000000,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.000000,0.000000,1.000000,0.000000,28.000000,0.000000,0.000000,1.000000,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,284,171,0.0,1.0,0.0,28.000000,0.000000,0.000000,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.000000,0.000000,1.000000,0.000000,28.000000,0.000000,0.000000,1.000000,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,284,172,0.0,1.0,0.0,28.000000,0.000000,0.000000,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.000000,0.000000,1.000000,0.000000,28.000000,0.000000,0.000000,1.000000,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,284,173,0.0,1.0,0.0,28.000000,0.000000,0.000000,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.000000,0.000000,1.000000,0.000000,28.000000,0.000000,0.000000,1.000000,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,284,174,0.0,1.0,0.0,28.000000,0.000000,0.000000,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.000000,0.000000,1.000000,0.000000,28.000000,0.000000,0.000000,1.000000,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,0.0,1.0,0.0,28.000000,0.0,0.0,1.0,1.032525e+06,1.032525e+06,1.032525e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416744,707,898,26.0,67.0,2.0,1940.333333,14.583333,1.500000,95.0,9.864667e+02,8.424347e+02,6.877667e+02,180.231022,0.464286,1.196429,0.035714,34.648810,0.260417,0.026786,1.696429,1.761548e+01,1.504348e+01,1.228155e+01,0.0,0.0,0.0,19.000000,0.0,0.0,1.0,2.300000e+00,2.300000e+00,1.466667e+00,2.0,3.0,1.0,60.000000,1.0,1.0,4.0,4.085000e+01,4.085000e+01,4.085000e+01,6.675223,0.023570,18.043008,0.631428,0.861703,0.187256,9.443649,0.374389,0.148313,0.851088,10.807529,9.327156,9.231422,4.853761
416745,708,898,105.0,165.0,3.0,3302.

### Vector solución

In [71]:
def variable_obj( df, ancla ):
    aux = df[['Ciclo_Estacion_Retiro', 'ancla', 'y']].loc[ df.ancla == (ancla+vobs) ].reset_index(drop=True)
    if ( ancla+vobs ) == 0:
        print("Voy en el ancla: ", ancla)
    return aux

In [72]:
%%time

y = pd.concat( map( lambda ancla: variable_obj( X_, ancla) , range(ancla_i, ancla_f+1) ), ignore_index=True )

CPU times: user 2.29 s, sys: 283 ms, total: 2.57 s
Wall time: 2.59 s


In [73]:
y['ancla'] = y['ancla'].astype('int32')
y['y'] = y['y'].astype('int32')

In [74]:
y

,Ciclo_Estacion_Retiro,ancla,y
0,001,202,19
1,002,202,6
2,005,202,14
3,006,202,15
4,007,202,26
...,...,...,...
376638,705,897,4
376639,706,897,1
376640,708,897,2
376641,710,897,6


In [75]:
# Hago checkpoint de y
y.to_parquet('../data/ecobici/y_ecobici.parquet')

In [77]:
%whos

Variable       Type                          Data/Info
------------------------------------------------------
X              DataFrame                            Ciclo_Estacion_Ret<...>416749 rows x 57 columns]
X_             DataFrame                            Ciclo_Estacion_Ret<...>379058 rows x 15 columns]
ancla_f        int64                         898
ancla_i        int64                         102
cat_hrs        DataFrame                                 hrs  ancla\n0<...>n\n[897 rows x 2 columns]
cols_X         list                          n=14
df             DataFrame                             Genero_Usuario  E<...>3893476 rows x 9 columns]
func_demanda   function                      <function func_demanda at 0x10cca1b20>
gc             module                        <module 'gc' (built-in)>
hr_f           int64                         897
hr_i           int64                         1
lst_var        list                          n=11
np             module                 

In [78]:
del X_
del df
del rango_fechas
del cat_hrs

gc.collect()

44

In [79]:
%whos

Variable       Type                          Data/Info
------------------------------------------------------
X              DataFrame                            Ciclo_Estacion_Ret<...>416749 rows x 57 columns]
ancla_f        int64                         898
ancla_i        int64                         102
cols_X         list                          n=14
func_demanda   function                      <function func_demanda at 0x10cca1b20>
gc             module                        <module 'gc' (built-in)>
hr_f           int64                         897
hr_i           int64                         1
lst_var        list                          n=11
np             module                        <module 'numpy' from '/Us<...>kages/numpy/__init__.py'>
os             module                        <module 'os' (frozen)>
pd             module                        <module 'pandas' from '/U<...>ages/pandas/__init__.py'>
reduce         builtin_function_or_method    <built-in function reduce>

In [80]:
X.shape, y.shape

((416749, 57), (376643, 3))

In [81]:
# Por si muere mi memoria puedo leer a partir de aquí los resultados
#X = pd.read_parquet( os.path.join('resultados','X_ecobici.parquet') )
#y = pd.read_parquet( os.path.join('resultados','y_ecobici.parquet') )

In [86]:
%%time
tad = X.merge( y, on = ['Ciclo_Estacion_Retiro','ancla'], how='inner' ).reset_index(drop=True)

CPU times: user 95.9 ms, sys: 162 ms, total: 258 ms
Wall time: 321 ms


In [87]:
tad.to_parquet('../data/ecobici/tad_viajes_G28.parquet')

In [88]:
pd.options.display.float_format = '{:,.3f}'.format

In [89]:
tad['y'].describe(percentiles=np.arange(0,1.01,.1))

count   376,643.000
mean          6.366
std           5.164
min           1.000
0%            1.000
10%           1.000
20%           2.000
30%           3.000
40%           4.000
50%           5.000
60%           6.000
70%           8.000
80%          10.000
90%          13.000
100%         66.000
max          66.000
Name: y, dtype: float64

In [90]:
tad.sort_values(['Ciclo_Estacion_Retiro','ancla'])

,Ciclo_Estacion_Retiro,ancla,v_F_tot_sum_100,v_M_tot_sum_100,v_ND_tot_sum_100,v_edad_prom_sum_100,v_pct_F_sum_100,v_pct_M_sum_100,v_personas_tot_sum_100,v_tiempo_viaje_max_sum_100,v_tiempo_viaje_mean_sum_100,v_tiempo_viaje_min_sum_100,v_tiempo_viaje_std_sum_100,v_F_tot_mean_100,v_M_tot_mean_100,v_ND_tot_mean_100,v_edad_prom_mean_100,v_pct_F_mean_100,v_pct_M_mean_100,v_personas_tot_mean_100,v_tiempo_viaje_max_mean_100,v_tiempo_viaje_mean_mean_100,v_tiempo_viaje_min_mean_100,v_F_tot_min_100,v_M_tot_min_100,v_ND_tot_min_100,v_edad_prom_min_100,v_pct_F_min_100,v_pct_M_min_100,v_personas_tot_min_100,v_tiempo_viaje_max_min_100,v_tiempo_viaje_mean_min_100,v_tiempo_viaje_min_min_100,v_F_tot_max_100,v_M_tot_max_100,v_ND_tot_max_100,v_edad_prom_max_100,v_pct_F_max_100,v_pct_M_max_100,v_personas_tot_max_100,v_tiempo_viaje_max_max_100,v_tiempo_viaje_mean_max_100,v_tiempo_viaje_min_max_100,v_tiempo_viaje_std_mean_100,v_tiempo_viaje_std_min_100,v_tiempo_viaje_std_max_100,v_F_tot_std_100,v_M_tot_std_100,v_ND_tot_std_100,v_edad_prom_std_100,v_pct_F_std_100,v_pct_M_std_100,v_personas_tot_std_100,v_tiempo_viaje_max_std_100,v_tiempo_viaje_mean_std_100,v_tiempo_viaje_min_std_100,v_tiempo_viaje_std_std_100,y
0,001,202,19.000,93.000,1.000,303.290,1.459,0.053,113.000,274.133,105.673,33.083,71.202,2.375,11.625,0.125,37.911,0.182,0.007,14.125,34.267,13.209,4.135,0.000,3.000,0.000,30.143,0.000,0.000,7.000,19.183,10.602,2.350,5.000,21.000,1.000,41.429,0.625,0.053,26.000,54.150,17.307,6.600,8.900,5.345,11.307,1.768,5.951,0.354,3.571,0.187,0.019,6.686,11.717,2.384,1.399,1.856,19
340,001,203,21.000,109.000,2.000,338.764,1.565,0.105,132.000,304.983,117.882,37.233,78.007,2.333,12.111,0.222,37.640,0.174,0.012,14.667,33.887,13.098,4.137,0.000,3.000,0.000,30.143,0.000,0.000,7.000,19.183,10.602,2.350,5.000,21.000,1.000,41.429,0.625,0.053,26.000,54.150,17.307,6.600,8.667,5.345,11.307,1.658,5.754,0.441,3.438,0.177,0.023,6.461,11.019,2.255,1.309,1.871,19
699,001,204,29.000,122.000,2.000,375.383,1.945,0.105,153.000,342.750,133.066,42.133,86.905,2.900,12.200,0.200,37.538,0.195,0.011,15.300,34.275,13.307,4.213,0.000,3.000,0.000,30.143,0.000,0.000,7.000,19.183,10.602,2.350,8.000,21.000,1.000,41.429,0.625,0.053,26.000,54.150,17.307,6.600,8.690,5.345,11.307,2.378,5.432,0.422,3.258,0.179,0.022,6.413,10.461,2.226,1.257,1.766,21
1061,001,205,30.000,125.000,2.000,414.633,2.195,0.105,157.000,374.017,148.549,48.483,97.899,2.727,11.364,0.182,37.694,0.200,0.010,14.273,34.002,13.504,4.408,0.000,3.000,0.000,30.143,0.000,0.000,4.000,19.183,10.602,2.350,8.000,21.000,1.000,41.429,0.625,0.053,26.000,54.150,17.307,6.600,8.900,5.345,11.307,2.328,5.853,0.405,3.133,0.171,0.021,6.973,9.965,2.211,1.355,1.813,4
1391,001,206,34.000,131.000,2.000,455.333,2.595,0.105,167.000,402.367,164.239,53.067,107.283,2.833,10.917,0.167,37.944,0.216,0.009,13.917,33.531,13.687,4.422,0.000,3.000,0.000,30.143,0.000,0.000,4.000,19.183,10.602,2.350,8.000,21.000,1.000,41.429,0.625,0.053,26.000,54.150,17.307,6.600,8.940,5.345,11.307,2.250,5.791,0.389,3.111,0.173,0.020,6.762,9.641,2.201,1.293,1.735,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375321,711,895,202.000,395.000,37.000,"3,229.718",28.945,6.256,634.000,"4,308.383","2,394.991","1,014.550","1,268.188",2.061,4.031,0.378,32.956,0.295,0.064,6.469,43.963,24.439,10.353,0.000,0.000,0.000,23.667,0.000,0.000,1.000,8.033,4.775,1.400,8.000,11.000,4.000,51.000,1.000,1.000,22.000,190.667,52.494,41.917,13.785,0.801,56.543,1.769,2.481,0.666,4.833,0.204,0.137,3.802,27.318,8.992,8.852,9.068,6
375980,711,896,202.000,394.000,37.000,"3,229.968",28.945,6.339,633.000,"4,295.600","2,390.349","1,030.817","1,255.335",2.061,4.020,0.378,32.959,0.295,0.065,6.459,43.833,24.391,10.519,0.000,0.000,0.000,23.667,0.000,0.000,1.000,8.033,4.775,1.400,8.000,11.000,4.000,51.000,1.000,1.000,22.000,190.66

In [91]:
import plotly.express as px

# Suponiendo que TAD es tu DataFrame y 'y' es la columna
fig = px.histogram(tad, x='y', title="Distribución de demanda de ecobicis")

# Mostrar la gráfica
fig.show()